In [2]:
from pyomo.environ import *
import numpy as pd
import pandas as pd 
import numpy as np 

In [3]:
# Load the arcs data from the excel sheet
List_arcs = pd.read_excel('dt2.xlsx', sheet_name="Arcs", index_col=False)

# Load the commodities data from the excel sheet
n_commodities = pd.read_excel('dt2.xlsx', sheet_name="Commodities", index_col=False)

In [4]:
# Create a Pyomo model
model = ConcreteModel()

# Define a set of nodes
n_nodes = Set(initialize=['1', '2', '3', '4', '5', '6', '7', '8'])
# Define the decision variables for the problem
model.x = Var(range(n_commodities), range(n_nodes), range(n_nodes), within=NonNegativeReals)

# Define the cost parameter for the problem
model.cost = Param(range(n_commodities), range(n_nodes), range(n_nodes), initialize=cost)

# Define the capacity parameter for the problem
model.capac = Param(range(n_commodities), range(n_nodes), range(n_nodes), initialize=capac)

# Define the supply parameter for the problem
model.supply = Param(range(n_commodities), range(n_nodes), initialize=supply)

# Define the demand parameter for the problem
model.demand = Param(range(n_commodities), range(n_nodes), initialize=demand)

# Define the objective function for the problem
def objective_rule(model):
    return sum(model.cost[i,j,k]*model.x[i,j,k] for i in range(n_commodities) for j in range(n_nodes) for k in range(n_nodes))

model.objective = Objective(rule=objective_rule, sense=minimize)

# Define the capacity constraints for the problem
def capac_rule(model, i, j, k):
    return model.x[i,j,k] <= model.capac[i,j,k]

model.capac_constraint = Constraint(range(n_commodities), range(n_nodes), range(n_nodes), rule=capac_rule)

# Define the supply constraints for the problem
def supply_rule(model, i, j):
    return sum(model.x[i,j,k] for k in range(n_nodes)) == model.supply[i,j]

model.supply_constraint = Constraint(range(n_commodities), range(n_nodes), rule=supply_rule)

# Define the demand constraints for the problem
def demand_rule(model, i, j):
    return sum(model.x[i,k,j] for k in range(n_nodes)) == model.demand[i,j]

model.demand_constraint = Constraint(range(n_commodities), range(n_nodes), rule=demand_rule)

# Solve the problem using a solver
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the optimal solution
print(results)


NameError: name 'n_commodities' is not defined

In [ ]:
print(model.objective())

for i in range(n_commodities):
    for j in range(n_nodes):
        for k in range(n_nodes):
            print(f'x[{i},{j},{k}] = {model.x[i,j,k].value}')
